In [1]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import loss_metrics
import conv_model
import plot_batch
import load_diagnosis
import icd_util

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [33]:
importlib.reload(icd_util)
importlib.reload(conv_model)
importlib.reload(loss_metrics)

<module 'loss_metrics' from '../libs/loss_metrics.py'>

In [15]:
icd_util.test()

5991 codes
9719 groups


In [5]:
ls /scr1/mimic

clinic/                initial_data_1449855/  metadata.hdf
empty_dats/            initial_data_1450598/  new_data/
files_lists/           initial_data_1451803/  new_data_raw/
headers/               initial_data_1454616/  sig_data.hdf
headers_matched_.pkl   initial_data_1455085/  waveforms/
headers_.pkl           initial_data_1455141/
initial_data_1449529/  metadata.csv


In [4]:
import os

In [8]:
os.mkdir

<function posix.mkdir(path, mode=511, *, dir_fd=None)>

In [5]:
# ! cp -r /scr1/mimic/initial_data_1455141 /scr1/mimic/initial_data

In [4]:
# mkdir /scr1/mimic/initial_data

In [1]:
# ! mv /scr1/mimic/initial_data /scr1/mimic/initial_data_1455753

In [2]:
%%time

path = '/scr1/mimic/initial_data/'
sig_data, metadata, diagnosis = initialize.load_initial_data(save_path=path)

CPU times: user 3min 21s, sys: 45.8 s, total: 4min 7s
Wall time: 4min 8s


In [3]:
diagnosis.shape

(48724, 84)

In [3]:
diagnosis.shape

(48724, 85)

In [4]:
diagnosis.shape

(48724, 86)

In [5]:
diagnosis.shape

(48692, 83)

In [3]:
diagnosis.shape

(48627, 86)

In [3]:
diagnosis.shape

(48894, 90)

In [3]:
diagnosis.shape

(48662, 103)

In [3]:
diagnosis.shape

(48894, 90)

In [4]:
diagnosis.shape

(51426, 97)

In [7]:
diagnosis

,code,038,070,150-159,160-165,162,190-199,200-208,244,250,250.4,...,769,780.3,785.51,785.52,789.5,799.0,799.02,995.9,995.91,995.92
subject_id,hadm_id,,,,,,,,,,,,,,,,,,,,,
3,145834,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,185777,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,107064,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,150750,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11,194540,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,176670,True,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,True,False,True
99991,151118,True,False,False,False,False,False,False,False,True,False,...,False,False,False,False,True,False,False,True,False,True
99992,197084,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [3]:
%%time
H = initialize.load_hypes()
parts = ['train', 'validation']

tensors, metadata, priors = initialize.run(H, parts)

Executing op Sum in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op NoOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LessEqual in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op All in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Equal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cumsum in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ConcatV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /

In [21]:
def get_offsets(indices):
    zeros = tf.zeros(shape=indices.shape[-1:], dtype='uint16')
    offsets = tf.stack([indices, zeros], axis=1)
    offsets = tf.expand_dims(tf.expand_dims(offsets, 1), 1)
    return offsets

def zfill(i, n):
    return tf.strings.substr('0'*n + tf.as_string(i), -n, n)

def get_examples(H, W, example):
    
    rand_int = lambda maxval: tf.random.uniform([1], 0, maxval, 'int32')[0]
    length = lambda tensor: tf.shape(tensor)[0]
    
    rec_ids = example.pop('rec_id')[0]
    seg_ids = example.pop('seg_id')
    
    i = rand_int(length(rec_ids))
    j = rand_int(length(seg_ids[i]))
    
    chunk_count = tf.cast(example.pop('chunk_count')[i][j], 'int32')
    chunk_id = rand_int(chunk_count)
    
    window_ids = tf.random.uniform(
        shape = [2**H['windows_per_chunk_log2']],
        minval = 2**H['window_size_log2'] * initialize.RESP_SCALE, 
        maxval = prepare_data.CHUNK_SIZE, 
        dtype = 'int32'
    )
    window_ids = tf.cast(window_ids, 'uint16')

    example['rec_id'] = rec_id = rec_ids[i]
    example['seg_id'] = seg_id = seg_ids[i][j]
    adc_gain  = example.pop('adc_gain')[i][j]
    baseline  = example.pop('baseline')[i][j]
    sig_index = example.pop('sig_index')[i][j]
    example['mask'] = sig_index > 0
    
    for key in ['diagnosis', 'age', 'weight', 'height']:
        example[key] = example[key][0][i]
    
    chunk_name = tf.as_string(rec_id) 
    chunk_name += '_' + zfill(seg_id, 4)
    chunk_name += '/' + zfill(chunk_id, 4)
        
    sigs_path = prepare_data.ROOT_SERIAL + chunk_name + '.tfrec'
    sigs = tf.io.read_file(sigs_path)
    sigs = tf.io.parse_tensor(sigs, out_type='int16')
    sigs.set_shape((prepare_data.CHUNK_SIZE, None))
    zeros = tf.zeros(shape=[prepare_data.CHUNK_SIZE, 1], dtype='int16')
    sigs = tf.concat([zeros, sigs], axis=1)
    sigs = tf.gather(sigs, tf.cast(sig_index, 'int32'), axis=1)
    sig_count = len(H['input_sigs'])
    sigs.set_shape((prepare_data.CHUNK_SIZE, sig_count))
    sigs = tf.cast(sigs - baseline, 'float32') / adc_gain
        
    dW = tf.cast(get_offsets(window_ids), 'int32')
    windows = tf.gather_nd(sigs, W + dW)
    metadata = {
        key: tf.stack([example[key]] * 2**H['windows_per_chunk_log2']) 
        for key in example
    }
    examples = {'sigs': windows, **metadata}
    examples = tf.data.Dataset.from_tensor_slices(examples)
    return examples

In [22]:
part = 'train'
n = next(iter(tensors[part].values())).shape[0]
key = 'batch_size' if part == 'train' else 'batch_size_validation'
batch_size = 2 ** H[key + '_log2']
buffer_size = 2 ** H['batch_buffer_size_log2'] if part == 'train' else 1
dataset = tf.data.Dataset.from_tensor_slices(tensors[part])
if part == 'validation' and H['batch_size_validation_log2'] > 9:
    dataset = dataset.repeat(2 ** (H['batch_size_validation_log2'] - 9))
dataset = dataset.shuffle(n)
window_index_matrix = data_pipeline.get_window_index_matrix(H)
dataset_b = dataset.interleave(
    partial(get_examples, H, window_index_matrix), 
    block_length = 1, 
    cycle_length = buffer_size * batch_size,
    num_parallel_calls = tf.data.experimental.AUTOTUNE,
)

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelInterleaveDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0


In [15]:
part = 'train'
n = next(iter(tensors[part].values())).shape[0]
key = 'batch_size' if part == 'train' else 'batch_size_validation'
batch_size = 2 ** H[key + '_log2']
buffer_size = 2 ** H['batch_buffer_size_log2'] if part == 'train' else 1
dataset = tf.data.Dataset.from_tensor_slices(tensors[part])
if part == 'validation' and H['batch_size_validation_log2'] > 9:
    dataset = dataset.repeat(2 ** (H['batch_size_validation_log2'] - 9))
dataset = dataset.shuffle(n)
window_index_matrix = data_pipeline.get_window_index_matrix(H)

dataset = dataset.map(partial(data_pipeline.get_examples, H, window_index_matrix))

S_train, S_val = H['input_sigs'], H['input_sigs_validation']
validation_mask = tf.constant([i in S_val for i in S_train], dtype='bool')

dataset = dataset.map(partial(data_pipeline.process_example, H, part, validation_mask))
dataset = dataset.filter(filter_example)
buffer_size *= batch_size * 2 ** H['windows_per_chunk_log2']
dataset = dataset.shuffle(buffer_size)

dataset = dataset.batch(batch_size).repeat()

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


AttributeError: in converted code:

    ../libs/data_pipeline.py:12 process_example  *
        sigs = example.pop('sigs')

    AttributeError: '_VariantDataset' object has no attribute 'pop'


In [23]:
list(dataset.take(1))

Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0


[{'seg_id': <tf.RaggedTensor [[3, 6, 7, 9, 11, 16, 24, 25, 29, 31, 33, 35, 37, 39, 43, 44, 46, 48, 50, 52, 55, 57, 59]]>,
  'chunk_count': <tf.RaggedTensor [[7, 27, 12, 1, 16, 418, 291, 75, 294, 59, 16, 79, 10, 82, 292, 204, 3, 264, 6, 55, 235, 1, 51]]>,
  'sig_index': <tf.RaggedTensor [[[0, 4, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0], [0, 4, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0], [0, 4, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0], [0, 4, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0], [0, 4, 2, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0], [0, 4, 0, 0, 0, 2, 1, 0, 0, 3, 0, 0, 0, 0, 0], [2, 4, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0], [2, 4, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0], [2, 4, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0], [2, 4,

In [24]:
list(dataset_b.take(1))

Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0


[{'sigs': <tf.Tensor: shape=(1024, 15), dtype=float32, numpy=
  array([[0.36862746,        nan,        nan, ...,        nan,        nan,
                 nan],
         [0.3647059 ,        nan,        nan, ...,        nan,        nan,
                 nan],
         [0.36078432,        nan,        nan, ...,        nan,        nan,
                 nan],
         ...,
         [0.27450982,        nan,        nan, ...,        nan,        nan,
                 nan],
         [0.27450982,        nan,        nan, ...,        nan,        nan,
                 nan],
         [0.27450982,        nan,        nan, ...,        nan,        nan,
                 nan]], dtype=float32)>,
  'diagnosis': <tf.Tensor: shape=(107,), dtype=int8, numpy=
  array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1

In [7]:
from functools import partial

In [28]:
f = partial(get_examples, H, window_index_matrix)
item = list(dataset.flat_map(f).take(1))
item

Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0


[{'sigs': <tf.Tensor: shape=(1024, 15), dtype=float32, numpy=
  array([[  nan,   nan,   nan, ...,   nan,   nan, 0.645],
         [  nan,   nan,   nan, ...,   nan,   nan, 0.705],
         [  nan,   nan,   nan, ...,   nan,   nan, 0.895],
         ...,
         [  nan,   nan,   nan, ...,   nan,   nan, 0.685],
         [  nan,   nan,   nan, ...,   nan,   nan, 0.665],
         [  nan,   nan,   nan, ...,   nan,   nan, 0.665]], dtype=float32)>,
  'diagnosis': <tf.Tensor: shape=(107,), dtype=int8, numpy=
  array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
          1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1,  1,  1, -1,  1,  1,
          1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,
          1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
          1, -1,  1,  1, -1], dtype=

In [ ]:
dataset.take(b)

In [ ]:
dataset = data_pipeline.build(H, tensors[part], part)

In [3]:
dataset = {part: data_pipeline.build(H, tensors[part], part) for part in parts}
model = conv_model.build(H, priors)

Executing op RaggedTensorToVariant in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RaggedTensorToVariant in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RaggedTensorToVariant in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RaggedTensorToVariant in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RaggedTensorToVariant in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousRandomSeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Range in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/d

In [8]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
signals (InputLayer)            [(None, 1024, 15)]   0                                            
__________________________________________________________________________________________________
mask (InputLayer)               [(None, 15)]         0                                            
__________________________________________________________________________________________________
tf_op_layer_ExpandDims_2 (Tenso [(None, 1024, 15, 1) 0           signals[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_ExpandDims (TensorF [(None, 1, 15)]      0           mask[0][0]                       
______________________________________________________________________________________________

In [4]:
model.fit(
    dataset['train'],
    validation_data = dataset['validation'],
    epochs = 2**20,
    steps_per_epoch = 2**H['steps_per_epoch_log2'],
    validation_steps = 2**H['validation_steps_log2'],
    validation_freq = range(0, 2**20, 2**H['validation_frequency_log2']),
    verbose = 1,
)

Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Train for 128 steps, validate for 128 steps
Epoch 1/1048576
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device 

KeyboardInterrupt: 

In [8]:
group_names = icd_util.load_group_strings()

In [12]:
import load_diagnosis

In [13]:
diag_ = load_diagnosis.load_raw()

In [17]:
diag_.reset_index()['code'].value_counts()

4019     20696
4280     13111
42731    12886
41401    12428
5849      9118
         ...  
71950        1
45374        1
37520        1
2799         1
75453        1
Name: code, Length: 6984, dtype: int64

In [2]:
root = '/scr1/mimic/'
clinic_file = lambda i: root + 'clinic/{}.csv'.format(i.upper())
diagnosis_info = pandas.read_csv(clinic_file('d_icd_diagnoses'))
diagnosis_info = diagnosis_info.drop(columns='ROW_ID').set_index('ICD9_CODE')
diagnosis_info.rename(columns={'LONG_TITLE': 'Condition'}, inplace=True)
diagnosis = pandas.read_csv(clinic_file('diagnoses_icd')).drop(columns='ROW_ID')
new_names = {i.upper(): i for i in ['subject_id', 'hadm_id']}
new_names['ICD9_CODE'] = 'code'
diagnosis = diagnosis[new_names].rename(columns=new_names)
diagnosis[:5]

,subject_id,hadm_id,code
0,109,172335,40301
1,109,172335,486
2,109,172335,58281
3,109,172335,5855
4,109,172335,4254


In [7]:
diagnosis = pandas.read_csv(clinic_file('diagnoses_icd'), dtype={'ICD9_CODE': 'string'})
new_names = {i.upper(): i for i in ['subject_id', 'hadm_id']}
new_names['ICD9_CODE'] = 'code'
diagnosis = diagnosis[new_names].rename(columns=new_names)

In [8]:
diagnosis.shape

(651047, 3)

In [9]:
diagnosis.drop_duplicates().shape

(650987, 3)

In [10]:
diagnosis

,subject_id,hadm_id,code
0,109,172335,40301
1,109,172335,486
2,109,172335,58281
3,109,172335,5855
4,109,172335,4254
...,...,...,...
651042,97503,188195,20280
651043,97503,188195,V5869
651044,97503,188195,V1279
651045,97503,188195,5275
